In [3]:
import pandas as pd
import pickle
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
data = pd.read_csv('dataset/data.csv')

# Select relevant columns
exog_columns = ['Age', 'BMI', 'UnusualBleeding', 'NumberofDaysofIntercourse', 'Breastfeeding', 'Numberpreg']
data = data[['CycleNumber', 'LengthofCycle'] + exog_columns].dropna()

# Set index
data.set_index('CycleNumber', inplace=True)

# Convert to numeric
data[exog_columns] = data[exog_columns].apply(pd.to_numeric, errors='coerce')
data['LengthofCycle'] = pd.to_numeric(data['LengthofCycle'], errors='coerce')

# Fill missing values with median
data.fillna(data.median(), inplace=True)

# Train-test split
train_size = int(len(data) * 0.8)
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]
train_target, test_target = train_data['LengthofCycle'], test_data['LengthofCycle']
train_exog, test_exog = train_data[exog_columns], test_data[exog_columns]

# Iterative ARIMA function
def iterative_arima(train_target, train_exog, test_target, test_exog, order=(2, 0, 2)):
    history_target = list(train_target)
    history_exog = list(train_exog.values)
    predictions = []
    
    for t in range(len(test_target)):
        # Fit ARIMA model on historical data
        model = ARIMA(history_target, exog=np.array(history_exog), order=order)
        model_fitted = model.fit()

        # Predict next step
        forecast = model_fitted.forecast(steps=1, exog=[test_exog.iloc[t].values])[0]
        
        # Debugging
        print(f"Step {t}: Exog = {test_exog.iloc[t].values}, Predicted Cycle Length = {forecast}")
        
        predictions.append(forecast)

        # Append new values to history for next iteration
        history_target.append(test_target.iloc[t])
        history_exog.append(test_exog.iloc[t].values)

        # Save the latest trained ARIMA model
        latest_model = model_fitted

    return predictions, latest_model

# Run iterative ARIMA
predictions, trained_model = iterative_arima(train_target, train_exog, test_target, test_exog)

# Evaluate performance
mae = mean_absolute_error(test_target, predictions)
rmse = np.sqrt(mean_squared_error(test_target, predictions))
print(f'MAE: {mae:.2f}, RMSE: {rmse:.2f}')

# Save trained ARIMA model
with open('iterative_arima_model_with_bmi.pkl', 'wb') as file:
    pickle.dump(trained_model, file)

print("✅ Iterative ARIMA model saved as iterative_arima_model_with_bmi.pkl")


d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Step 0: Exog = [30.5        24.13850309  0.         12.          0.          2.        ], Predicted Cycle Length = 33.19410946497279


d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Step 1: Exog = [30.5        24.13850309  0.          9.          0.          2.        ], Predicted Cycle Length = 30.860865324606234


d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Step 330: Exog = [30.5        24.13850309  0.          7.          0.          2.        ], Predicted Cycle Length = 28.695440012594407


d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Step 331: Exog = [30.5        24.13850309  0.          3.          0.          2.        ], Predicted Cycle Length = 27.849519448905042
Step 332: Exog = [30.5        24.13850309  0.          4.          0.          2.        ], Predicted Cycle Length = 30.637854992187844
MAE: 2.56, RMSE: 3.72
✅ Iterative ARIMA model saved as iterative_arima_model_with_bmi.pkl


d:\GitHub\floofy\py_backend\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
